In [1]:
import pandas as pd
import os
from os.path import join, isfile
from bokeh.palettes import inferno
from bokeh.plotting import figure, save
from bokeh.io import output_file, show
from bokeh.layouts import column, widgetbox
from datetime import datetime
from bokeh.models import HoverTool, ColumnDataSource, Legend, CustomJS
from bokeh.models.widgets import Dropdown

In [2]:
def build_dataset(fpath, ext='.csv'):
    files = [join(fpath, f) for f in os.listdir(fpath) if isfile(join(fpath, f))]
    dfs = list()
    for f in files:
        dfs.append(pd.read_csv(f))
    df = pd.concat(dfs)
    df.columns = [col[0].upper() + col[1:] for col in df.columns]
    df['Time'] = pd.to_datetime(df['Time'])
    df.sort_values('Time', ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.drop_duplicates('Time', keep='first', inplace=True)
    return df


def build_visualization(symbol, fpath='Data/', outpath='Visualizations/', bad_vars=['Time', 'Volume']):
    symbol = symbol.lower()
    datasets = dict()
    datasets['Daily'] = build_dataset(fpath + f'{symbol}/{symbol}_daily/')
    datasets['Intraday'] = build_dataset(fpath + f'{symbol}/{symbol}_intraday/')
    datasets['Sentiment'] = build_dataset(fpath + f'{symbol}/{symbol}_sentiment/')
    figs = list()
    for dataset in datasets.keys():
        s = figure(plot_width=1600, plot_height=500, x_axis_type="datetime")
        s.title.text = f'Data from {dataset}'
        columns = [c for c in list(datasets[dataset].columns) if c not in bad_vars]
        colors = inferno(len(columns))
        for col, color in zip(columns, colors):
            line = s.line(x='Time', y=col, source=datasets[dataset], line_width=2, color=color, alpha=0.8, legend_label=col)
            s.add_tools(HoverTool(renderers=[line], tooltips=[('date', '@Time{%F}'), (f'{col}',f'@{col}'), ('Volume', f'@Volume')], formatters={'Time': 'datetime', f'{col}': f'numeral', 'Volume': f'numeral'},mode='mouse'))
        s.legend.location = "top_left"
        s.legend.click_policy="hide"
        figs.append(s)
    now = datetime.now()
    output_file(outpath + f'{symbol}_viz_{now.year}_{now.month}_{now.day}.html', title=f'Plots for {symbol}')
    c = column(*figs)
    return c
#     show(c)
#     save(c)

In [3]:
symbol_df = pd.read_csv('symbol_map.csv')
symbol_list = list(symbol_df.symbol)

In [4]:
plots = dict()
for symbol in symbol_list:
    try:
        plots[symbol] = build_visualization(symbol)
    except:
        print(f'Viz for {symbol} failed.') 
    else:
        print(f'Viz for {symbol} succeeded.')

Viz for aapl succeeded.
Viz for twtr succeeded.
Viz for tsla succeeded.
Viz for nflx succeeded.
Viz for fb succeeded.
Viz for msft succeeded.
Viz for nvda succeeded.
Viz for ge succeeded.
Viz for amd succeeded.
Viz for bac succeeded.
Viz for ko succeeded.
Viz for dis succeeded.
Viz for gpro succeeded.
Viz for sbux succeeded.
Viz for baba succeeded.
Viz for amzn succeeded.
Viz for googl succeeded.
Viz for brk.b failed.
Viz for tcehy succeeded.
Viz for v succeeded.
Viz for wmt succeeded.
Viz for voo succeeded.
Viz for spy succeeded.
Viz for intc succeeded.
Viz for t succeeded.
Viz for vz succeeded.
Viz for csco succeeded.
Viz for crm succeeded.


In [7]:
html_body = str()
from datetime import datetime
now = datetime.now()
for symbol in plots.keys():
    html_body += f'<a href="Visualizations/{symbol}_viz_{now.year}_{now.month}_{now.day}.html"> {symbol} </a><br>'

In [8]:
with open('main_page.html', 'w', encoding='utf-8') as f:
    f.write(f'<html>\n<body>{html_body}</body></html>')